<a href="https://colab.research.google.com/github/hotorch/SST2_fine_tuning/blob/master/SST2_data_with_huggingface_transformer(remark_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## huggingface transformers 

In [97]:
!pip install transformers

In [98]:
from transformers import BertModel, BertTokenizer # load language model & tokenizer

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

huggingface에서 https://huggingface.co/transformers/model_doc/bert.html 를 참조

In [99]:
bert = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [100]:
# tokenizer test
text = 'The BERT model was proposed in BERT'

print(tokenizer(text))

len(tokenizer(text))

{'input_ids': [101, 1996, 14324, 2944, 2001, 3818, 1999, 14324, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


3

In [101]:
# encode -> list type
# encode 함수에 스페셜 토큰을 보고 싶지 않을 때에는 add_special_tokens = False 활용
tokenizer.encode(text)

[101, 1996, 14324, 2944, 2001, 3818, 1999, 14324, 102]

In [102]:
# decode
tokenizer.decode(tokenizer.encode(text))

'[CLS] the bert model was proposed in bert [SEP]'

#### 조금 더 살펴보기

In [103]:
input_ids = tokenizer.encode(text)
input_ids_tensor = torch.tensor(input_ids, dtype = torch.long).unsqueeze(0).cuda()
bert = BertModel.from_pretrained('bert-base-uncased').cuda()

In [104]:
embeddings = bert(input_ids_tensor)
embeddings[0]
print(embeddings[0].shape) # |batch_size, # of tokens, bert dimension|

torch.Size([1, 9, 768])


#### Load Data(SST-2)

https://nlp.stanford.edu/sentiment/index.html



In [105]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [106]:
path = "drive/My Drive/SST-2/"
os.listdir(path)

['train.tsv', 'dev.tsv', 'test.tsv']

In [107]:
train = pd.read_csv(path + 'train.tsv', delimiter = '\t')[:1000] # 오래 걸리기 때문에 일부분만 활용함
# dev = pd.read_csv('dev.tsv', delimiter = '\t')
# test = pd.read_csv('test.tsv', delimiter = '\t')

In [45]:
# 데이터 문장 길이가 다르기 때문에 [PAD] token을 활용해야함, 또한 최대 길이도 정해야함
print(train['sentence'].apply(lambda x: tokenizer.encode(x))[0])
print(train['sentence'].apply(lambda x: tokenizer.encode(x))[1]) 

[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102]
[101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015, 102]


#### Define Max Length & fix dimension

In [108]:
MAX_LEN = train['sentence'].apply(lambda x: len(x)).max() # 250
train['sentence'] = train['sentence'].apply(lambda x: tokenizer.encode(x))
# insert [pad]
padded_ids = np.array([sentence + [0] * (MAX_LEN - len(sentence)) 
					   for sentence in train['sentence']])
print(padded_ids.shape)

(1000, 250)


In [109]:
# 필요한 문장만을 활용하기 위해 attention mask 활용
attention_mask = np.where(np.array(padded_ids) != 0, 1, 0)
# tensor로 변환하기 
padded_ids_tensor = torch.tensor(padded_ids, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype= torch.long)
output_tensor = torch.tensor(train[['label']].values, dtype = torch.long) 

#### Define Model

In [125]:
class SentimentClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = bert
        self.linear = nn.Linear(768, 2)

        for parameter in self.bert.parameters():
            parameter.requires_grad = False # bert안에 있는 parameter 건드리지 않기, 오로지 nn.linear만 학습

    def forward(self, input_ids, attention_mask):
        # input_ids dimension : |batch_size, MAX_LEN, embedding_dim|
        input_ids = self.bert(input_ids, attention_mask= attention_mask)[0]
        # ouput dimension : |batch_size, MAX_LEN, embedding_dim|
        input_ids = input_ids[:,0,:]
        return self.linear(input_ids)

#### Define DataLoader

In [114]:
from torch.utils.data import Dataset, DataLoader
# 데이터 길이, 데이터 인덱스 던져줬을 때 어떤 것을 리턴할지만 고려하기

class SST2(Dataset):
    def __init__(self, 
               input_ids,
               attention_mask,
               output):
        super().__init__()
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.output = output
  
    def __len__(self): 
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.attention_mask[index], self.output[index]

In [126]:
net = SentimentClassifier().cuda()
optimizer = optim.SGD(net.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss().cuda()
EPOCHS = 100

In [116]:
train_dataset = SST2(padded_ids_tensor[:700],
                     attention_mask_tensor[:700],
                     output_tensor[:700])

valid_dataset = SST2(padded_ids_tensor[700:],
                     attention_mask_tensor[700:],
                     output_tensor[700:])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=True)

In [127]:
import tqdm

for epoch in tqdm.tqdm_notebook(range(EPOCHS)):
	for input_ids, attention_mask, output in tqdm.tqdm_notebook(train_dataloader):
		predictions = net(input_ids.cuda(), attention_mask.cuda())
		loss = loss_fn(predictions, output.cuda().squeeze())
		loss.backward()
		optimizer.step()
		optimizer.zero_grad()

	with torch.no_grad():
		num_correct = 0
		for input_ids, attention_mask, output in tqdm.tqdm_notebook(valid_dataloader):
			predictions = net(input_ids.cuda(), attention_mask.cuda())
			loss = loss_fn(predictions, output.cuda().squeeze())

			num_correct += (predictions.max(dim=1)[1] == output.cuda().squeeze()).sum().item()

		accuracy = num_correct / len(valid_dataloader) * 8

		print(accuracy)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]



34.73684210526316



45.68421052631579


...


54.31578947368421



53.05263157894737



54.526315789473685



55.1578947368421



53.473684210526315

